In [1]:
import requests
import json
import pandas as pd
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from sklearn.metrics import precision_score, recall_score

In [2]:
import nltk
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_excel('30_sample_penyakit.xlsx')
df = df.head(5)
df

,Kode,keluhan_utama,PARAPHRASE KELUHAN UTAMA,jalannya_penyakit,PARAPHRASE JALANNYA PENYAKIT,pemeriksaan_penunjang,hasil_laboratorium,diagnosa_utama,kode_icd_utama,diagnosa_sekunder_1,kode_icd_sekunder_1,kondisi_pulang,obat_waktu_pulang
0,A001,pasien datang dengan keluhan demam terutama pa...,Pasien mengalami demam pada waktu malam. Sebel...,sudah minum obat belum ada perubahan,Sudah minum obat dan ada perubahan.,"td: 120/80 hr: 100x t: 38,1",widal: (+) 1/160,Demam tifoid,A01.0,Tidak terdata,Tidak terdata,Hidup,floxifar 2x1 erlamol 3x1 metyl 2x1 rani 2x1
1,A002,bab cair lebih dari 6x sejak 1 hari lalu denga...,Pasien mengalami buang air besar cair lebih da...,- Pemeriksaan penunjang yang positif : TD 140/...,- Pemeriksaan penunjang yang positif : TD 140/...,"TD 140/70 N 88x R 21x S36,8 Kepala, thorax : d...",Tidak terdata,Other gastroenteritis and colitis of infectiou...,A09,Tidak terdata,Tidak terdata,inj ranitidin antasida 3x1 pct 31 lopamid prn,inj ranitidin antasida 3x1 pct 31 lopamid prn
2,A003,PASIEN DATANG DENGAN KELUHAN DEMAM 6 HARI SMRS...,Pasien datang dengan keluhan demam selama 6 ha...,SUDAH MINUM OBAT BELUM ADA PERUBAHAN Pemeriksa...,SUDAH MINUM OBAT BELUM ADA PERUBAHAN Pemeriksa...,"BB: 40KG T: 39,8 PTEKIE (+)",TR: 142RB,Dengue fever [classical dengue],A90,Tidak terdata,Tidak terdata,Hidup Obat-obatan waktu pulang/nasihat : PASIE...,PASIEN MENOLAK RAWAT INAP PCT 3X1 DEXTEEM 3X1 ...
3,B001,demam sejak 3 hari lalu disertai muncul bruntu...,Pasien menderita demam disertai munculnya brun...,sudah berobat namun beelum ada perbaikan Pemer...,Sudah berobat tapi belum ada perubahan. TD = 1...,TD 150/80 N 88x R 20x S 38 Status generalis db...,Tidak terdata,Zoster [herpes zoster],B02,"Fever, unspecified",R50.9,injeksi ketorolac pct 3x1 asiklovir 5x400mg vi...,injeksi ketorolac pct 3x1 asiklovir 5x400mg vi...
4,i001,pasien datang dengan keluhan nyeri dada sejak ...,Pasien nyeri dada sejak 4 jam lalu seperti ter...,nyeri berulang dengan durasi 20 menit TD : 125...,nyeri berulang dengan durasi 20 menit TD : 125...,tidak ada,Tidak terdata,hipertensi urgensi,i16,Tidak terdata,Tidak terdata,amlodipin 10 mg 1x1 captopril 25 mg 2x1 injeks...,amlodipin 10 mg 1x1 captopril 25 mg 2x1 injeks...


In [4]:
OPENROUTER_API_KEY="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"

def query_deepseek(prompt):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "deepseek/deepseek-chat:free",
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    try:
        result = response.json()
        return result["choices"][0]["message"]["content"]  # Extract LLM output
    except:
        return "ERROR"

In [45]:
df["llm_response"] = df["PARAPHRASE KELUHAN UTAMA"].apply(query_deepseek)

In [54]:
cosine_similarities = []
precisions = []
recalls = []
bleu_scores = []
meteor_scores = []
# rouge_l_scores = []

In [55]:
for index, row in df.iterrows():
    ground_truth = str(row["diagnosa_utama"]).lower()  
    llm_output = str(row["llm_response"]).lower()  

    if not ground_truth or not llm_output:  
        cosine_similarities.append(-1)
        precisions.append(-1)
        recalls.append(-1)
        bleu_scores.append(-1)
        meteor_scores.append(-1)
        # rouge_l_scores.append(-1)
        continue

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([ground_truth, llm_output])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    cosine_similarities.append(cosine_sim)

    truth_words = set(ground_truth.split())
    response_words = set(llm_output.split())

    true_positives = len(truth_words & response_words)
    false_positives = len(response_words - truth_words)
    false_negatives = len(truth_words - response_words)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    precisions.append(precision)
    recalls.append(recall)

    bleu = sentence_bleu([ground_truth.split()], llm_output.split())
    bleu_scores.append(bleu)

    meteor = meteor_score([ground_truth.split()], llm_output.split())
    meteor_scores.append(meteor)

    # rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    # rouge_score_l = rouge.score(ground_truth, llm_output)["rougeL"].fmeasure
    # rouge_l_scores.append(rouge_score_l)

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower o

In [57]:
df["cosine_similarity"] = cosine_similarities[:len(df)]
df["precision"] = precisions[:len(df)]
df["recall"] = recalls[:len(df)]
df["bleu_score"] = bleu_scores[:len(df)]
df["meteor_score"] = meteor_scores[:len(df)]
# df["rouge_l"] = rouge_l_scores[:len(df)]

df[["PARAPHRASE KELUHAN UTAMA", "diagnosa_utama", "llm_response", "bleu_score", "cosine_similarity", "precision", "recall", "meteor_score"]]

,PARAPHRASE KELUHAN UTAMA,diagnosa_utama,llm_response,bleu_score,cosine_similarity,precision,recall,meteor_score
0,Pasien mengalami demam pada waktu malam. Sebel...,Demam tifoid,"Berdasarkan gejala yang Anda sebutkan, yaitu d...",4.545727e-232,0.193615,0.006579,0.500000,0.018116
1,Pasien mengalami buang air besar cair lebih da...,Other gastroenteritis and colitis of infectiou...,"Berdasarkan gejala yang Anda jelaskan, kemungk...",5.296346e-232,0.018217,0.009346,0.125000,0.022624
2,Pasien datang dengan keluhan demam selama 6 ha...,Dengue fever [classical dengue],"Berdasarkan gejala yang Anda deskripsikan, pas...",5.106573e-232,0.136049,0.007937,0.250000,0.025253
3,Pasien menderita demam disertai munculnya brun...,Zoster [herpes zoster],Berdasarkan gejala dan hasil pemeriksaan yang ...,4.388531e-232,0.089488,0.005848,0.333333,0.015432
4,Pasien nyeri dada sejak 4 jam lalu seperti ter...,hipertensi urgensi,"Berdasarkan gejala yang Anda deskripsikan, yai...",0.000000e+00,0.000000,0.000000,0.000000,0.000000


In [58]:
print(df['llm_response'][0])

Berdasarkan gejala yang Anda sebutkan, yaitu demam yang terjadi pada malam hari, disertai mual, muntah, dan menggigil, ada beberapa kemungkinan diagnosis yang dapat dipertimbangkan. Namun, untuk diagnosis yang akurat, diperlukan pemeriksaan lebih lanjut oleh dokter, termasuk pemeriksaan fisik, tes laboratorium, dan riwayat medis lengkap.

Beberapa kemungkinan penyakit yang dapat menyebabkan gejala tersebut:

1. **Malaria**:  
   - Gejala khas malaria termasuk demam yang bersifat periodik (terjadi pada waktu tertentu, seperti malam hari), menggigil, mual, muntah, dan nyeri otot.  
   - Penyakit ini disebabkan oleh parasit *Plasmodium* yang ditularkan melalui gigitan nyamuk *Anopheles*.  

2. **Demam Berdarah Dengue (DBD)**:  
   - Gejalanya meliputi demam tinggi mendadak, sakit kepala parah, nyeri sendi dan otot, mual, muntah, dan terkadang ruam kulit.  
   - DBD disebabkan oleh virus dengue yang ditularkan melalui gigitan nyamuk *Aedes aegypti*.  

3. **Tifoid (Demam Tifoid)**:  
   - 